In [6]:
import pandas as pd
import numpy as np
import time
from pandas import Series, DataFrame
data = pd.read_csv('song_tag_map.csv')

all_tags = []

start = time.time()

for index, row in data.iterrows():
    all_tags = list(set(all_tags + eval(row['tags'])))
    if((index % 5000) == 0):
        print(index)
        print(time.time()- start)
        
all_tags_dict = dict(zip(all_tags,range(len(all_tags))))

print('converting started')

data_id = DataFrame({'song' : [],'tag' : []})

for index, row in data.iterrows():
    song = row['songs']
    tags_str = eval(row['tags'])
    tags_int = []
    for tag_name in tags_str:
        tags_int.append(all_tags_dict[tag_name])
    data_id = data_id.append({'song' : song, 'tag' : tags_int}, ignore_index = True)
    if((index % 5000) == 0):
        print(index)

0
0.16445183753967285
5000
4.771586894989014
10000
13.964149951934814
15000
25.148191928863525
20000
37.6056649684906
25000
53.43792200088501
30000
68.1614179611206
35000
82.34769582748413
40000
95.1989848613739
45000
108.10333681106567
50000
121.49280095100403
55000
134.62274503707886
60000
148.17681980133057
65000
161.66417407989502
70000
175.2822368144989
75000
188.38796997070312
80000
201.44546675682068
85000
214.16468286514282
90000
226.8247480392456
95000
239.58642196655273
100000
252.27044892311096
105000
264.9835340976715
110000
277.6696879863739
115000
290.6493229866028
120000
303.4115397930145
125000
316.11339497566223
130000
328.95254707336426
135000
341.7074737548828
140000
354.5733458995819
145000
367.35041403770447
150000
380.17896604537964
155000
392.9428849220276
160000
405.7819130420685
165000
418.7216889858246
170000
431.519966840744
175000
444.4668319225311
180000
457.29790592193604
185000
470.1928539276123
190000
483.0915038585663
195000
495.9406650066376
200000
508

In [7]:
import csv

data_id.to_csv("tag_id.csv")

with open('tag_id_dict.csv', 'w') as f:
    for key in all_tags_dict.keys():
        f.write("%s,%s\n"%(key,all_tags_dict[key]))

In [96]:
len(all_tags)

28742

In [6]:
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re

import numpy as np
import pandas as pd

from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from konlpy.tag import Twitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from pandas.plotting import register_matplotlib_converters
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [9]:
valid = pd.read_json('val.json', typ = 'frame')

In [30]:
n = 100

In [34]:
valid

,id,like_cnt,plylst_title,songs,tags,updt_date
0,118598,1675,,"[373313, 151080, 275346, 696876, 165237, 52593...",[],2019-05-27 14:14:33.000
1,131447,1,앨리스테이블,[],[],2014-07-16 15:24:24.000
2,51464,62,,"[529437, 516103, 360067, 705713, 226062, 37089...",[],2008-06-21 23:26:22.000
3,45144,20,,"[589668, 21711, 570151, 320043, 13930, 599327,...",[],2017-10-30 18:15:43.000
4,79929,20,,"[672718, 121924, 102694, 683657, 201558, 38511...",[],2017-02-07 11:40:42.000
5,138538,2,,"[173033, 138237, 63140, 15733, 461780, 308519,...","[비, 분위기, 컨트리, 서정적인]",2019-06-12 17:43:35.000
6,127575,2,,"[227044, 424672, 515592, 22408, 57893, 153714,...",[],2011-10-28 11:21:51.000
7,115317,3,,"[54457, 423722, 337682, 91214, 644980, 556444,...",[],2014-01-26 20:00:30.000
8,80810,127,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},[],[스트레스],2017-02-09 17:33:45.000
9,142007,0,기분 좋은 재즈와 함께 만드는 달달한 하루,[],[],2015-06-22 09:11:02.000


In [32]:
valid.loc[n]['songs']

[498228,
 697026,
 154753,
 366524,
 223723,
 674425,
 336951,
 148979,
 11849,
 35139,
 684580,
 205033,
 117188,
 31769,
 383928,
 248867,
 585562,
 583086,
 534578,
 588693,
 112379,
 601944,
 265752,
 449245]

In [33]:
def listofsimilarsongs(songs_know):
    # ------------ input -------------
    # songs_know : playlist에 있는 songs들을 담은 list
    
    
    candidate_songs = dict()
    
    for song in songs_know:
        
        #함수 이름 앵버가 짜면 바꿔야 함
        toadd = similar_songs(song)
        
        # candidate_songs에서 얼만큼 언급되었는지 세어준다
        for it in toadd:
            if(it in candidate_songs):
                candidate_songs[it] += 1
            else: #처음 등장한 녀석
                candidate_songs[it] = 1
    
    candidate_songs = dict(sorted(candidate_songs.items(), reverse=True, key=lambda item: item[1]))

    if(len(candidate_songs) < 100):
        print("candidate les than 100")
    
    real_candidate = list(candidate_songs.keys())[0:99]
    
    #for debugging 대충 몇 개정도씩 들어있는지?
    print(sum(list(candidate_songs.values()))/ len(list(candidate_songs.values())))
    
    return real_candidate

In [35]:
# Numpy Matrix
# Column : Each song
# Row : Shingles



# Instance variables :
# n : the number of total songs
# s : the number of signatures
# r : the number of signatures in a single band
# b : the number of bands (s // r)
# perm : permutated np array
# mhdata : minhashed (signature array) 
#          row : signatures / columns : songs
# ---------------------------------------------------------------
class LSH() :
    import numpy as np 
    # Initializer
    # Input
    # data : pandas dataframe for song - shingles
    #        row : songs / column : array of tags for each song
    # sig_n : the number of signatures
    # band_r : the number of signatures in a single band
    # bucket_sz : bucket size for LSH
    # -----------------------------------------------------------
    def __init__(self, data, sig_n, band_r) :
        self.data = data
        self.n = data.shape[0]
        self.s = sig_n
        self.r = band_r
        self.bucket_sz = min(500, 1 << self.r)
        if self.s%self.r != 0 :
            print("The number of signatures is not divided by the number of band_r.")
            print("Warning : Remainder signatures will be ignored")
            self.s = self.s - self.s%self.r
        self.b  = self.s // self.r
        self.song_dic = dict(zip(self.data.iloc[:,0],range(len(self.data.iloc[:,0]))))   #index might change

        
    # permutation
    # save random shuffle of 1:n 
    def permute(self, tag_n) :
        self.perm = np.random.permutation(tag_n)
        
    # This function uses permutation to hash, outputs the minimum value
    # tag_n : Total number of tags (required for permutation)
    # This algorithm could be improved using
    # https://datalab.snu.ac.kr/~ukang/courses/19S-DM/L5-lsh.pdf page 42 later
    def minhash(self, tag_n = None) :
        # result matrix
        # row : signatures / columns : songs
        sig_mat = np.zeros((self.s, self.n))
        for sig_num in range(self.s) :
            self.permute(tag_n)
            new_sig = [min(np.take(self.perm,i)) for i in self.data.iloc[:,1]]  # index 1 could be changed later
            sig_mat[sig_num, :] = new_sig
        self.mhdata = sig_mat
    
    # This function locally hashes the signature vector (of size self.r)
    def local_hash(self, sig_vec) :
        return hash(str(sig_vec)) % self.bucket_sz
    
    # Computes locally sensitive hashing algorithm (put into buckets)
    # Bucket_mtx : row - bands, column - bucket for each band 
    def lsh(self) :
        mh_t = np.transpose(self.mhdata)
        bucket_mtx = np.zeros((self.b, self.bucket_sz, 1))
        for band_i in self.b :
            curr = band_i * self.r 
            hashed = [self.local_hash(col[curr:curr+self.r]) for col in mh_t]
            for ind, item in enumerate(hashed, start=0): :
                bucket_mtx[band_i][item] = np.append(bucket_mtx[band_i][item], ind)
        
        self.bucket_mtx = bucket_mtx
    
    # Outputs the array of similar songs w.r.t given song
    def sim_items(self, song) :
        sim_item = np.array([])
        song_sig = self.mhdata[:,self.song_dic[song]]
        for band_i in self.b :
            curr = band_i * self.r
            curr_hash_val = self.local_hash(song_sig[curr:curr+self.r])
            bucket_for_this = self.bucket_mtx[band_i][curr_hash_val]
            sim_item = numpy.union1d(sim_item, bucket_for_this)
        return np.take(self.data.iloc[:, 0], sim_item)

SyntaxError: invalid syntax (<ipython-input-35-765d7dfdfaa0>, line 71)